In [1]:
import pandas as pd
import glob

path = r'./reviews/' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df

,id,asin.original,asin.variant,review_data,date.date,date.unix,name,rating,title,review,verified_purchase
0,R30N0JN6JA1MRQ,B082F94M3S,B082F94M3S,"Reviewed in the United States on August 3, 2021","August 3, 2021",1627970400,Anna P,3,Produces yogurt that’s too sour and tart,The product came out too sour for my taste.The...,True
1,R15FIYNUIYKMRP,B082F94M3S,B082F94M3S,"Reviewed in the United States on July 21, 2021","July 21, 2021",1626847200,Maya,1,Didn't work. Sour and inedible.,Tried it twice. Once according to the directio...,True
2,R390P68FSXHAYC,B082F94M3S,B082F94M3S,"Reviewed in the United States on June 14, 2021","June 14, 2021",1623650400,justadumbblonde,5,Makes gourmet yogurt with beginner skills,"I'd never made my own yogurt before, so I writ...",True
3,R3BPTBIV53BU25,B082F94M3S,B001E0VA8G,"Reviewed in the United States on June 1, 2021","June 1, 2021",1622527200,manisha patel,5,yummy taste,I like it. Its my second order and I love this...,True
4,R3CLAKMEP4ASKG,B082F94M3S,B001E0VA8G,"Reviewed in the United States on May 20, 2021","May 20, 2021",1621490400,Jonesy,5,It consistently makes good tasting yogurt,"This is a little pricey for my budget, but I s...",True
...,...,...,...,...,...,...,...,...,...,...,...
10781,R2FQ0WZYUXTZ9M,B07LFBC64X,NaN,"Reviewed in the United States on January 16, 2021","January 16, 2021",1610780400,Missy Mae,3,not great,yoghurt lacks flavor. Compared to LA Fermere y...,True
10782,R260AVQ4RJIKW5,B07LFBC64X,NaN,"Reviewed in the United States on November 17, ...","November 17, 2020",1605596400,ashley,5,Best Quality Yogurt,Literally my favorite yogurt! Only wish they c...,True
10783,R28FCOAKK1OKKR,B07LFBC64X,NaN,"Reviewed in the United States on July 16, 2020","July 16, 2020",1594879200,Natalia,5,Gimme more!!,Yum yum yum. I can’t get enough - my new go-to...,True
10784,R2QBRN76JF3IZS,B07LFBC64X,NaN,"Reviewed in the United States on April 26, 2020","April 26, 2020",1587880800,Nancy,4,Not bad,"This yogurt was okay, nothing special. It come...",True


In [2]:
import json
import spacy

nlp = spacy.load("en_core_web_sm")

reviews = []
titles = []

for index, row in df.iterrows():
    title = str(row['title'])
    review = str(row['review'])
    
    titles.append(title + " " + review)
    
    reviews.append({
        "title": title,
        "review": review,
    })

In [3]:
len(reviews)

10786

In [11]:
%env GOOGLE_APPLICATION_CREDENTIALS=./key.json

import json
from google.cloud import language_v1
client = language_v1.LanguageServiceClient()

type_ = language_v1.Document.Type.PLAIN_TEXT
sentiments = []

# Edit the 100 to the total docs, however it'll cost me $$$ to do sentiment analysis on all 10k reviews
for title in titles[0:100]:
    document = {"content": title, "type_": type_}
    encoding_type = language_v1.EncodingType.UTF8
    
    response = client.analyze_sentiment(request = {'document': document, 'encoding_type': encoding_type})
    sentiments.append({
        "text": title,
        "magnitude": response.document_sentiment.magnitude,
        "score": response.document_sentiment.score
    })
        
file = open("./sentiments.json", "w")
file.write(json.dumps(sentiments, indent=4))

env: GOOGLE_APPLICATION_CREDENTIALS=./key.json


552317

In [3]:
from pprint import pprint
import nltk
from nltk.corpus import stopwords
import collections
import json

nltk.download('stopwords')

words = []
stpwrds = stopwords.words('english')
count = 0
bad = []

with open('./sentiments.json') as json_file:
    data = json.load(json_file)

    for sentiment in data:
        if(sentiment['score'] < 0):
            bad.append(sentiment['text'])
            count = count + 1
            for word in sentiment['text'].split():
                if(word not in stpwrds):
                    words.append(word)

def CountFrequency(arr):
    return collections.Counter(arr).most_common()

freq = CountFrequency(words)
freq

# for thing in bad:
#     if("taste" in thing):
#         print(thing, "\n")

UNRELIABLE I discovered this brand, first at Himalayan Institute PA. The yogurt is tangy ( how everyone outside United States, believes the yogurt should be) and Smooth. So good that I had to go to the kitchen and ask what starter they use. I ordered it and for a whole year, I had the most amazing yogurt. Easy, economical, tart, perfect consistency. Much btter than The European Yogurt you buy at Trader Joe's.I ordered it again the next year from the same Amazon distributor. Not a single one in the package worked! And I hopelessly tried to experiment with increasing the incubation time, the cooling time, mixing two packagage... Not a single one worked. I feel that if there was a possibility to buy the product from the company itself rather than Amazon, the issue could have been resolved. But it is not possible. Or... Is it just an unreliable company? I am very disappointed. 

Failed flavor Did not care for the flavor at all. I like Chobani god their nutritional values and because they’r

[nltk_data] Downloading package stopwords to /Users/jrmy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
